<a href="https://colab.research.google.com/github/alinaalborova/russian_idioms_processing/blob/main/MICE_multiBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Idiom Type and Token Classification

Based on [MICE: Mining Idioms with Contextual Embeddings](https://arxiv.org/pdf/2008.05759.pdf) by  Škvorc et al.


In [ ]:
!pip install transformers
!pip install tensor2tensor

     |████████████████████████████████| 2.3MB 3.9MB/s 
     |████████████████████████████████| 901kB 39.4MB/s 
     |████████████████████████████████| 3.3MB 39.2MB/s 
     |████████████████████████████████| 1.5MB 3.9MB/s 
     |████████████████████████████████| 655kB 33.3MB/s 
     |████████████████████████████████| 5.6MB 23.5MB/s 
     |████████████████████████████████| 194kB 44.2MB/s 
     |████████████████████████████████| 358kB 40.4MB/s 
     |████████████████████████████████| 81kB 7.8MB/s 
     |████████████████████████████████| 368kB 34.5MB/s 
     |████████████████████████████████| 983kB 44.8MB/s 
     |████████████████████████████████| 686kB 42.3MB/s 
     |████████████████████████████████| 368kB 25.9MB/s 
     |████████████████████████████████| 256kB 39.7MB/s 
  Created wheel for pypng: filename=pypng-0.0.20-cp37-none-any.whl size=67163 sha256=8db9423431ddf80297b32b8417cb394e132140c084e50b5317a133b45b4f0eab
  Stored in directory: /root/.cache/pip/wheels/41/6b/ef/0493b536b6d472

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
import tensorflow as tf
import transformers as ppb
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from tensorflow.keras.layers import Bidirectional, LSTM, Dense, Activation, TimeDistributed, Masking, GRU
from tensorflow.keras import Sequential
from tensorflow.keras.utils import to_categorical
from ast import literal_eval
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from collections import Counter
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score

## Dataset

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
dataset_vnc_dir = '/content/drive/MyDrive/ВКР/Sense Disambiguation Corpus/VNCs_Annotated.csv'
data_vnc = pd.read_csv(dataset_vnc_dir )
data_vnc.head(2)

,Idiom Normal,Idiom Inflected,Label,Example
0,бить карту,бил карту,0,Он бил карту за картой и загребал золото и кре...
1,бить карту,бил карту,0,"Ермолов держал карты, сощуря правый глаз; ког..."


In [ ]:
data_vnc.Label.value_counts()

1    455
0    438
Name: Label, dtype: int64

In [ ]:
data_vnc.shape

(893, 4)

In [ ]:
len(data_vnc['Idiom Normal'].value_counts())

51

In [ ]:
len(data_anc['Idiom Inflected'].value_counts())

180

In [ ]:
dataset_anc_dir = '/content/drive/MyDrive/ВКР/Sense Disambiguation Corpus/ANCs_Annotated.csv'
data_anc = pd.read_csv(dataset_anc_dir )
data_anc.head(2)

,Idiom Normal,Idiom Inflected,Label,Example
0,избитая дорога,избитой дороге,0,"С бурной быстротой, возможной только в сновид..."
1,избитая дорога,избитой дороге,0,"Как почтовый возок на избитой дороге, прыгает..."


In [ ]:
data_anc['Idiom Normal'].value_counts()

больное место                 57
правая рука                   56
болевая точка                 52
нож острый                    49
путеводная звезда             48
лавровый венок                44
бедный родственник            41
зелёная улица                 38
тяжёлая рука                  38
ваш брат                      34
вавилонское столпотворение    30
наша сестра                   28
пороховая бочка               27
дальний прицел                25
вторая ступень                23
заблудшая овца                23
старый воробей                22
красная бумажка               21
синяя птица                   20
другой разговор               18
долгая песня                  18
старый гриб                   16
чёрная кость                  15
девичья кожа                  12
маковое зерно                 10
избитая дорога                10
музейная редкость             10
куриная голова                 9
ободранная кошка               9
чернильная строка              3
Name: Idio

In [ ]:
len(data_anc['Idiom Normal'].value_counts())

81

In [ ]:
data['Label'].value_counts()

0    856
1    843
Name: Label, dtype: int64

In [ ]:
data.shape

(1699, 4)

In [ ]:
data = pd.concat([data_vnc, data_anc], ignore_index=True)
data.head(2)

,Idiom Normal,Idiom Inflected,Label,Example
0,бить карту,бил карту,0,Он бил карту за картой и загребал золото и кре...
1,бить карту,бил карту,0,"Ермолов держал карты, сощуря правый глаз; ког..."


In [ ]:
data.tail(2)

,Idiom Normal,Idiom Inflected,Label,Example
1697,бедный родственник,бедными родственниками,0,"Проходя мимо церквей, я вижу иногда человека,..."
1698,бедный родственник,бедных родственниках,0,"[Егор Дмитрич Глумов, муж] У молодой женщин..."


In [ ]:
data.shape

(1699, 4)

## Load BERT

In [ ]:
model_class, tokenizer_class, pretrained_weights = (ppb.TFBertModel, ppb.BertTokenizer, 'bert-base-multilingual-cased')

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
BERT_model = model_class.from_pretrained(pretrained_weights)

Some layers from the model checkpoint at bert-base-multilingual-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-multilingual-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
def pad_sentence(tokenized, max_len):
  print('\nPadding/truncating all sentences to %d values...' % max_len)
  print('\nPadding token: "{:}", ID: {:}'.format(tokenizer.pad_token, tokenizer.pad_token_id))

  # Pad our input tokens with value 0.
  # "post" indicates that we want to pad and truncate at the end of the sequence,
  # as opposed to the beginning.
  input_ids = pad_sequences(tokenized, maxlen=max_len, dtype="long", 
                            value=0, truncating="post", padding="post")
  print('\nDone.')
  return input_ids


def create_att_masks(input_ids):
  # Create attention masks
  attention_masks = []

  # For each sentence...
  for sent in input_ids:
      
      # Create the attention mask.
      #   - If a token ID is 0, then it's padding, set the mask to 0.
      #   - If a token ID is > 0, then it's a real token, set the mask to 1.
      att_mask = [int(token_id > 0) for token_id in sent]
      
      # Store the attention mask for this sentence.
      attention_masks.append(att_mask)
  attention_masks = np.array(attention_masks)    
  return attention_masks

def extract_full_embeddings(output):
  last_hidden_states = output[0] # lhs for all sentences
  extracted = []
  for i, el in enumerate(last_hidden_states): #for each sentence...
    extracted.append(last_hidden_states[i]) 
  return extracted

def use_batches(padded, masked):
  full_embeddings = []

  NUM_OF_IDIOMS_initial = len(masked)
  BATCH_SIZE = 200  # Using larger batch might kill the session when embedding contexts
  NUM_OF_IDIOMS = NUM_OF_IDIOMS_initial
  i = 0

  while NUM_OF_IDIOMS > 0:
    print(i)
    NUM_OF_IDIOMS -= BATCH_SIZE
    print('NUM_OF_IDIOMS -= BATCH_SIZE', NUM_OF_IDIOMS)
    if i < NUM_OF_IDIOMS_initial - BATCH_SIZE:
      output_batch = BERT_model(padded[i:i+BATCH_SIZE], attention_mask = masked[i:i+BATCH_SIZE])
    else:
      output_batch = BERT_model(padded[i:NUM_OF_IDIOMS_initial], attention_mask = masked[i:NUM_OF_IDIOMS_initial])
    i += BATCH_SIZE
    embeddings_batch = extract_full_embeddings(output_batch)
    full_embeddings.append(embeddings_batch)

  full_embeddings_all = []
  for batch in full_embeddings:
    for sentence in batch:
      full_embeddings_all.append(sentence)

  return full_embeddings_all


## Define RNN

In [ ]:
MAX_SEQUENCE_LEN = 150
VECTOR_DIM = 768
NUM_CLASSES = 2

def build_model():
  model = Sequential()
  model.add(Masking(mask_value=0., input_shape=(MAX_SEQUENCE_LEN,VECTOR_DIM)))
  forward_layer = GRU(10, return_sequences=False, dropout=0.5)
  backward_layer = GRU(10, return_sequences=False, dropout=0.5,
                      go_backwards=True)
  model.add(Bidirectional(forward_layer, backward_layer=backward_layer,
                        input_shape=(MAX_SEQUENCE_LEN,VECTOR_DIM)))
  model.add(Dense(NUM_CLASSES))
  model.add(Activation('softmax'))

  model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
  print('compiled model')
  return model

## All Idioms

In [ ]:
tokenized_idioms = data['Idiom Inflected'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True, truncation=True, max_length=50)))
tokenized_contexts = data['Example'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True, truncation=True, max_length=500)))
max_idiom_length = max([len(sen) for sen in tokenized_idioms])
max_context_length = max([len(sen) for sen in tokenized_contexts])
print('Max idiom length: ', max_idiom_length)
print('Max context length: ', max_context_length)

padded_idioms = pad_sentence(tokenized_idioms, max_idiom_length+5)
padded_contexts = pad_sentence(tokenized_contexts, max_context_length+20)
print(len(padded_idioms))
print(len(padded_contexts))

masked_idioms = create_att_masks(padded_idioms)
masked_contexts = create_att_masks(padded_contexts)
print(len(masked_idioms))
print(len(masked_contexts))

Max idiom length:  11
Max context length:  234

Padding/truncating all sentences to 16 values...

Padding token: "[PAD]", ID: 0

Done.

Padding/truncating all sentences to 254 values...

Padding token: "[PAD]", ID: 0

Done.
1699
1699
1699
1699


In [ ]:
embedded_contexts = use_batches(padded_contexts, masked_contexts)
len(embedded_contexts)

0
NUM_OF_IDIOMS -= BATCH_SIZE 1499
200
NUM_OF_IDIOMS -= BATCH_SIZE 1299
400
NUM_OF_IDIOMS -= BATCH_SIZE 1099
600
NUM_OF_IDIOMS -= BATCH_SIZE 899
800
NUM_OF_IDIOMS -= BATCH_SIZE 699
1000
NUM_OF_IDIOMS -= BATCH_SIZE 499
1200
NUM_OF_IDIOMS -= BATCH_SIZE 299
1400
NUM_OF_IDIOMS -= BATCH_SIZE 99
1600
NUM_OF_IDIOMS -= BATCH_SIZE -101


1699

In [ ]:
data

,Idiom Normal,Idiom Inflected,Label,Example
0,бить карту,бил карту,0,Он бил карту за картой и загребал золото и кре...
1,бить карту,бил карту,0,"Ермолов держал карты, сощуря правый глаз; ког..."
2,бить карту,бил карту,0,"Он сгреб кучку золота, прибавил к ней на глаз..."
3,бить карту,бил карту,0,Передо мною все мелькала бледная улыбка банко...
4,бить карту,бейте карту,1,"Бейте карту: или я, или вы комендантом в замк..."
...,...,...,...,...
1694,бедный родственник,бедными родственниками,1,"И мы, россиянцы, будем чувствовать себя на эт..."
1695,бедный родственник,бедными родственниками,1,"Скорее, бедными родственниками являются именн..."
1696,бедный родственник,бедными родственниками,0,Александр II с детских лет привык считать бед...
1697,бедный родственник,бедными родственниками,0,"Проходя мимо церквей, я вижу иногда человека,..."


In [ ]:
labels = to_categorical(data.Label)

In [ ]:
X_train, X_test = train_test_split(embedded_contexts, test_size=0.2, random_state=34)
Y_train, Y_test = train_test_split(labels, test_size=0.2, random_state=34)

### Classifier

In [ ]:
model_all = build_model()

compiled model


In [ ]:
model_all.fit(np.asarray(X_train), Y_train, batch_size=8, epochs=10)

Epoch 1/10
170/170 [==============================] - 49s 239ms/step - loss: 0.6055 - accuracy: 0.6909
Epoch 2/10
170/170 [==============================] - 41s 239ms/step - loss: 0.5049 - accuracy: 0.7586
Epoch 3/10
170/170 [==============================] - 41s 239ms/step - loss: 0.4621 - accuracy: 0.7822
Epoch 4/10
170/170 [==============================] - 40s 237ms/step - loss: 0.4232 - accuracy: 0.8124
Epoch 5/10
170/170 [==============================] - 40s 237ms/step - loss: 0.3935 - accuracy: 0.8227
Epoch 6/10
170/170 [==============================] - 40s 236ms/step - loss: 0.3521 - accuracy: 0.8492
Epoch 7/10
170/170 [==============================] - 40s 236ms/step - loss: 0.3214 - accuracy: 0.8668
Epoch 8/10
170/170 [==============================] - 40s 236ms/step - loss: 0.2876 - accuracy: 0.8749
Epoch 9/10
170/170 [==============================] - 40s 236ms/step - loss: 0.2714 - accuracy: 0.8756
Epoch 10/10
170/170 [==============================] - 40s 236ms/step - l

In [ ]:
model_all.evaluate(np.asarray(X_test), Y_test)

11/11 [==============================] - 4s 106ms/step - loss: 0.6364 - accuracy: 0.7912


[0.6363776922225952, 0.7911764979362488]

In [ ]:
preds_all = model_all.predict(np.array(X_test))
f1_score(np.argmax(preds_all, axis=1), np.argmax(Y_test, axis=1))

0.7942028985507246

In [ ]:
 n_classes = 3

fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(Y_test_anc[:, i], preds_anc[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

## VNC

In [ ]:
tokenized_idioms_vnc = data_vnc['Idiom Inflected'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True, truncation=True, max_length=50)))
tokenized_contexts_vnc = data_vnc['Example'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True, truncation=True, max_length=500)))
max_idiom_length_vnc = max([len(sen) for sen in tokenized_idioms_vnc])
max_context_length_vnc = max([len(sen) for sen in tokenized_contexts_vnc])
print('Max idiom length: ', max_idiom_length_vnc)
print('Max context length: ', max_context_length_vnc)

padded_idioms_vnc = pad_sentence(tokenized_idioms_vnc, max_idiom_length_vnc+5)
padded_contexts_vnc = pad_sentence(tokenized_contexts_vnc, max_context_length_vnc+20)
print(len(padded_idioms_vnc))
print(len(padded_contexts_vnc))

Max idiom length:  11
Max context length:  234

Padding/truncating all sentences to 16 values...

Padding token: "[PAD]", ID: 0

Done.

Padding/truncating all sentences to 254 values...

Padding token: "[PAD]", ID: 0

Done.
893
893


In [ ]:
masked_idioms_vnc = create_att_masks(padded_idioms_vnc)
masked_contexts_vnc = create_att_masks(padded_contexts_vnc)
print(len(masked_idioms_vnc))
print(len(masked_contexts_vnc))

893
893


In [ ]:
embedded_contexts_vnc = use_batches(padded_contexts_vnc, masked_contexts_vnc)
len(embedded_contexts_vnc)

0
NUM_OF_IDIOMS -= BATCH_SIZE 693
200
NUM_OF_IDIOMS -= BATCH_SIZE 493
400
NUM_OF_IDIOMS -= BATCH_SIZE 293
600
NUM_OF_IDIOMS -= BATCH_SIZE 93
800
NUM_OF_IDIOMS -= BATCH_SIZE -107


893

In [ ]:
labels_vnc = to_categorical(data_vnc.Label)
X_train_vnc, X_test_vnc = train_test_split(embedded_contexts_vnc, test_size=0.2, random_state=34)

In [ ]:
Y_train_vnc, Y_test_vnc = train_test_split(labels_vnc, test_size=0.2, random_state=34)

### Classifier

In [ ]:
model_vnc = build_model()

compiled model


In [ ]:
model_vnc.fit(np.asarray(X_train_vnc), Y_train_vnc, batch_size=8, epochs=10)

Epoch 1/10
90/90 [==============================] - 27s 227ms/step - loss: 0.6189 - accuracy: 0.6597
Epoch 2/10
90/90 [==============================] - 20s 227ms/step - loss: 0.4896 - accuracy: 0.7885
Epoch 3/10
90/90 [==============================] - 21s 231ms/step - loss: 0.4268 - accuracy: 0.8025
Epoch 4/10
90/90 [==============================] - 21s 230ms/step - loss: 0.3694 - accuracy: 0.8445
Epoch 5/10
90/90 [==============================] - 21s 229ms/step - loss: 0.3265 - accuracy: 0.8599
Epoch 6/10
90/90 [==============================] - 21s 230ms/step - loss: 0.2719 - accuracy: 0.8838
Epoch 7/10
90/90 [==============================] - 20s 228ms/step - loss: 0.2343 - accuracy: 0.9034
Epoch 8/10
90/90 [==============================] - 21s 229ms/step - loss: 0.2227 - accuracy: 0.9090
Epoch 9/10
90/90 [==============================] - 21s 229ms/step - loss: 0.1841 - accuracy: 0.9300
Epoch 10/10
90/90 [==============================] - 21s 230ms/step - loss: 0.1505 - accura

In [ ]:
model_vnc.evaluate(np.asarray(X_test_vnc), Y_test_vnc)

6/6 [==============================] - 3s 96ms/step - loss: 0.4989 - accuracy: 0.8324


[0.49890008568763733, 0.832402229309082]

In [ ]:
preds_vnc = model_vnc.predict(np.array(X_test_vnc))
f1_score(np.argmax(preds_vnc, axis=1), np.argmax(Y_test_vnc, axis=1))

0.8514851485148516

## ANC

In [ ]:
tokenized_contexts_anc = data_anc['Example'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True, truncation=True, max_length=500)))
max_context_length_anc = max([len(sen) for sen in tokenized_contexts_anc])
print('Max context length: ', max_context_length_anc)

padded_contexts_anc = pad_sentence(tokenized_contexts_anc, max_context_length_anc+20)
print(len(padded_contexts_anc))

Max context length:  221

Padding/truncating all sentences to 241 values...

Padding token: "[PAD]", ID: 0

Done.
806


In [ ]:
masked_contexts_anc = create_att_masks(padded_contexts_anc)
print(len(masked_contexts_anc))

806


In [ ]:
embedded_contexts_anc = use_batches(padded_contexts_anc, masked_contexts_anc)
len(embedded_contexts_anc)

0
NUM_OF_IDIOMS -= BATCH_SIZE 606
200
NUM_OF_IDIOMS -= BATCH_SIZE 406
400
NUM_OF_IDIOMS -= BATCH_SIZE 206
600
NUM_OF_IDIOMS -= BATCH_SIZE 6
800
NUM_OF_IDIOMS -= BATCH_SIZE -194


806

In [ ]:
labels_anc = to_categorical(data_anc.Label)
X_train_anc, X_test_anc = train_test_split(embedded_contexts_anc, test_size=0.2, random_state=34)
Y_train_anc, Y_test_anc = train_test_split(labels_anc, test_size=0.2, random_state=34)

### Classifier

In [ ]:
model_anc = build_model()

compiled model


In [ ]:
model_anc.fit(np.asarray(X_train_anc), Y_train_anc, batch_size=8, epochs=10)

Epoch 1/10
81/81 [==============================] - 25s 219ms/step - loss: 0.6665 - accuracy: 0.5978
Epoch 2/10
81/81 [==============================] - 18s 223ms/step - loss: 0.5584 - accuracy: 0.7314
Epoch 3/10
81/81 [==============================] - 18s 219ms/step - loss: 0.4962 - accuracy: 0.7702
Epoch 4/10
81/81 [==============================] - 18s 221ms/step - loss: 0.4379 - accuracy: 0.8090
Epoch 5/10
81/81 [==============================] - 18s 218ms/step - loss: 0.3870 - accuracy: 0.8323
Epoch 6/10
81/81 [==============================] - 18s 218ms/step - loss: 0.3415 - accuracy: 0.8634
Epoch 7/10
81/81 [==============================] - 20s 253ms/step - loss: 0.2965 - accuracy: 0.8882
Epoch 8/10
81/81 [==============================] - 18s 222ms/step - loss: 0.2672 - accuracy: 0.8975
Epoch 9/10
81/81 [==============================] - 18s 219ms/step - loss: 0.2328 - accuracy: 0.9022
Epoch 10/10
81/81 [==============================] - 18s 219ms/step - loss: 0.2094 - accura

In [ ]:
model_anc.evaluate(np.asarray(X_test_anc), Y_test_anc)

6/6 [==============================] - 3s 83ms/step - loss: 0.6992 - accuracy: 0.7840


[0.6991873383522034, 0.7839506268501282]

In [ ]:
preds_anc = model_anc.predict(np.array(X_test_anc))

In [ ]:
f1_score(np.argmax(preds_anc, axis=1), np.argmax(Y_test_anc, axis=1))

0.7798742138364779

# Not Present in the Training Set

## Split

In [ ]:
data_anc['Idiom Normal'].value_counts()

больное место                 57
правая рука                   56
болевая точка                 52
нож острый                    49
путеводная звезда             48
лавровый венок                44
бедный родственник            41
зелёная улица                 38
тяжёлая рука                  38
ваш брат                      34
вавилонское столпотворение    30
наша сестра                   28
пороховая бочка               27
дальний прицел                25
заблудшая овца                23
вторая ступень                23
старый воробей                22
красная бумажка               21
синяя птица                   20
долгая песня                  18
другой разговор               18
старый гриб                   16
чёрная кость                  15
девичья кожа                  12
маковое зерно                 10
избитая дорога                10
музейная редкость             10
куриная голова                 9
ободранная кошка               9
чернильная строка              3
Name: Idio

In [ ]:
test_ancs = ['вавилонское столпотворение', 'ободранная кошка', 'наша сестра', 'пороховая бочка', 'дальний прицел', 'заблудшая овца', 'красная бумажка']

In [ ]:
data_anc_test = data_anc.loc[data_anc['Idiom Normal'].isin(test_ancs)]
data_anc_test.shape

(163, 4)

In [ ]:
data_anc_train = data_anc.loc[~data_anc['Idiom Normal'].isin(test_ancs)]
data_anc_train.shape

(643, 4)

In [ ]:
163/806

0.2022332506203474

In [ ]:
data_vnc['Idiom Normal'].value_counts()

бросать тень               53
сесть на мель              47
пускать корни              41
окунуться с головой        31
отвести глаза              29
давит грудь                29
поставить на колени        29
выступить на сцену         27
приложить руку             27
снимать шляпу              27
поднять на ноги            27
положить голову            25
пахнет порохом             23
вырвать с корнем           23
точить нож                 22
преградить дорогу          22
вильнуть хвостом           21
сидеть на печи             20
открыть глаза              20
умывать руки               19
открывать Америку          19
плести кружева             18
взваливать на плечи        17
имей глаза                 17
поддать жару               16
разбить лед                16
чесать затылок             16
прокладывать дорогу        14
бросать перо               14
прижать хвост              13
давать сдачу               13
катить бочку               12
прищемить хвост            12
поймать на

In [ ]:
test_vncs = ['точить нож', 'преградить дорогу', 'вильнуть хвостом', 'сидеть на печи', 
             'открыть глаза', 'умывать руки', 'открывать Америку', 'плести кружева', 
             'положить голову']

In [ ]:
data_vnc_test = data_vnc.loc[data_vnc['Idiom Normal'].isin(test_vncs)]
data_vnc_test.shape

(186, 4)

In [ ]:
data_vnc_train = data_vnc.loc[~data_vnc['Idiom Normal'].isin(test_ancs)]
data_vnc_train.shape

(893, 4)

In [ ]:
186/893

0.20828667413213886

In [ ]:
data_all_train = pd.concat([data_vnc_train, data_anc_train], ignore_index=True)
data_all_train.head(2)

,Idiom Normal,Idiom Inflected,Label,Example
0,бить карту,бил карту,0,Он бил карту за картой и загребал золото и кре...
1,бить карту,бил карту,0,"Ермолов держал карты, сощуря правый глаз; ког..."


In [ ]:
data_all_test = pd.concat([data_vnc_test, data_anc_test], ignore_index=True)
data_all_test.head(2)

,Idiom Normal,Idiom Inflected,Label,Example
0,открывать Америку,открывать Америку,1,С тех пор от него можно услышать: «Хватит кич...
1,открывать Америку,открывать Америку,0,"Впечатление было такое, что мы на судне Колум..."


## Embed

### VNC

In [ ]:
tokenized_idioms_vnc_train_not_present = data_vnc_train['Idiom Inflected'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True, truncation=True, max_length=50)))
tokenized_contexts_vnc_train_not_present = data_vnc_train['Example'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True, truncation=True, max_length=500)))
max_idiom_length_vnc_train_not_present = max([len(sen) for sen in tokenized_idioms_vnc_train_not_present])
max_context_length_vnc_train_not_present = max([len(sen) for sen in tokenized_contexts_vnc_train_not_present])
print('Max idiom length: ', max_idiom_length_vnc_train_not_present)
print('Max context length: ', max_context_length_vnc_train_not_present)

padded_idioms_vnc_train_not_present = pad_sentence(tokenized_idioms_vnc_train_not_present, max_idiom_length_vnc_train_not_present+5)
padded_contexts_vnc_train_not_present = pad_sentence(tokenized_contexts_vnc_train_not_present, max_context_length_vnc_train_not_present+20)
print(len(padded_idioms_vnc_train_not_present))
print(len(padded_contexts_vnc_train_not_present))

Max idiom length:  11
Max context length:  234

Padding/truncating all sentences to 16 values...

Padding token: "[PAD]", ID: 0

Done.

Padding/truncating all sentences to 254 values...

Padding token: "[PAD]", ID: 0

Done.
893
893


In [ ]:
masked_idioms_vnc_train_not_present = create_att_masks(padded_idioms_vnc_train_not_present)
masked_contexts_vnc_train_not_present = create_att_masks(padded_contexts_vnc_train_not_present)
print(len(masked_idioms_vnc_train_not_present))
print(len(masked_contexts_vnc_train_not_present))

893
893


In [ ]:
embedded_idioms_vnc_train_not_present = use_batches(padded_idioms_vnc_train_not_present, masked_idioms_vnc_train_not_present)
len(embedded_idioms_vnc_train_not_present)

0
NUM_OF_IDIOMS -= BATCH_SIZE 693
200
NUM_OF_IDIOMS -= BATCH_SIZE 493
400
NUM_OF_IDIOMS -= BATCH_SIZE 293
600
NUM_OF_IDIOMS -= BATCH_SIZE 93
800
NUM_OF_IDIOMS -= BATCH_SIZE -107


893

In [ ]:
embedded_contexts_vnc_train_not_present = use_batches(padded_contexts_vnc_train_not_present, masked_contexts_vnc_train_not_present)
len(embedded_contexts_vnc_train_not_present)

0
NUM_OF_IDIOMS -= BATCH_SIZE 693
200
NUM_OF_IDIOMS -= BATCH_SIZE 493
400
NUM_OF_IDIOMS -= BATCH_SIZE 293
600
NUM_OF_IDIOMS -= BATCH_SIZE 93
800
NUM_OF_IDIOMS -= BATCH_SIZE -107


893

In [ ]:
labels_vnc_train_not_present = to_categorical(data_vnc_train.Label)

In [ ]:
tokenized_idioms_vnc_test_not_present = data_vnc_test['Idiom Inflected'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True, truncation=True, max_length=50)))
tokenized_contexts_vnc_test_not_present = data_vnc_test['Example'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True, truncation=True, max_length=500)))
max_idiom_length_vnc_test_not_present = max([len(sen) for sen in tokenized_idioms_vnc_test_not_present])
max_context_length_vnc_test_not_present = max([len(sen) for sen in tokenized_contexts_vnc_test_not_present])
print('Max idiom length: ', max_idiom_length_vnc_test_not_present)
print('Max context length: ', max_context_length_vnc_test_not_present)

padded_idioms_vnc_test_not_present = pad_sentence(tokenized_idioms_vnc_test_not_present, max_idiom_length_vnc_test_not_present+5)
padded_contexts_vnc_test_not_present = pad_sentence(tokenized_contexts_vnc_test_not_present, max_context_length_vnc_test_not_present+20)
print(len(padded_idioms_vnc_test_not_present))
print(len(padded_contexts_vnc_test_not_present))

masked_idioms_vnc_test_not_present = create_att_masks(padded_idioms_vnc_test_not_present)
masked_contexts_vnc_test_not_present = create_att_masks(padded_contexts_vnc_test_not_present)
print(len(masked_idioms_vnc_test_not_present))
print(len(masked_contexts_vnc_test_not_present))

embedded_idioms_vnc_test_not_present = use_batches(padded_idioms_vnc_test_not_present, masked_idioms_vnc_test_not_present)
len(embedded_idioms_vnc_test_not_present)

embedded_contexts_vnc_test_not_present = use_batches(padded_contexts_vnc_test_not_present, masked_contexts_vnc_test_not_present)
len(embedded_contexts_vnc_test_not_present)

Max idiom length:  10
Max context length:  170

Padding/truncating all sentences to 15 values...

Padding token: "[PAD]", ID: 0

Done.

Padding/truncating all sentences to 190 values...

Padding token: "[PAD]", ID: 0

Done.
186
186
186
186
0
NUM_OF_IDIOMS -= BATCH_SIZE -14
0
NUM_OF_IDIOMS -= BATCH_SIZE -14


186

In [ ]:
labels_vnc_test_not_present = to_categorical(data_vnc_test.Label)

### ANC

In [ ]:
tokenized_idioms_anc_train_not_present = data_anc_train['Idiom Inflected'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True, truncation=True, max_length=50)))
tokenized_contexts_anc_train_not_present = data_anc_train['Example'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True, truncation=True, max_length=500)))
max_idiom_length_anc_train_not_present = max([len(sen) for sen in tokenized_idioms_anc_train_not_present])
max_context_length_anc_train_not_present = max([len(sen) for sen in tokenized_contexts_anc_train_not_present])
print('Max idiom length: ', max_idiom_length_anc_train_not_present)
print('Max context length: ', max_context_length_anc_train_not_present)

padded_idioms_anc_train_not_present = pad_sentence(tokenized_idioms_anc_train_not_present, max_idiom_length_anc_train_not_present+5)
padded_contexts_anc_train_not_present = pad_sentence(tokenized_contexts_anc_train_not_present, max_context_length_anc_train_not_present+20)
print(len(padded_idioms_anc_train_not_present))
print(len(padded_contexts_anc_train_not_present))

masked_idioms_anc_train_not_present = create_att_masks(padded_idioms_anc_train_not_present)
masked_contexts_anc_train_not_present = create_att_masks(padded_contexts_anc_train_not_present)
print(len(masked_idioms_anc_train_not_present))
print(len(masked_contexts_anc_train_not_present))

embedded_idioms_anc_train_not_present = use_batches(padded_idioms_anc_train_not_present, masked_idioms_anc_train_not_present)
len(embedded_idioms_anc_train_not_present)

embedded_contexts_anc_train_not_present = use_batches(padded_contexts_anc_train_not_present, masked_contexts_anc_train_not_present)
len(embedded_contexts_anc_train_not_present)

Max idiom length:  10
Max context length:  221

Padding/truncating all sentences to 15 values...

Padding token: "[PAD]", ID: 0

Done.

Padding/truncating all sentences to 241 values...

Padding token: "[PAD]", ID: 0

Done.
643
643
643
643
0
NUM_OF_IDIOMS -= BATCH_SIZE 443
200
NUM_OF_IDIOMS -= BATCH_SIZE 243
400
NUM_OF_IDIOMS -= BATCH_SIZE 43
600
NUM_OF_IDIOMS -= BATCH_SIZE -157
0
NUM_OF_IDIOMS -= BATCH_SIZE 443
200
NUM_OF_IDIOMS -= BATCH_SIZE 243
400
NUM_OF_IDIOMS -= BATCH_SIZE 43
600
NUM_OF_IDIOMS -= BATCH_SIZE -157


643

In [ ]:
labels_anc_train_not_present = to_categorical(data_anc_train.Label)

In [ ]:
tokenized_idioms_anc_test_not_present = data_anc_test['Idiom Inflected'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True, truncation=True, max_length=50)))
tokenized_contexts_anc_test_not_present = data_anc_test['Example'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True, truncation=True, max_length=500)))
max_idiom_length_anc_test_not_present = max([len(sen) for sen in tokenized_idioms_anc_test_not_present])
max_context_length_anc_test_not_present = max([len(sen) for sen in tokenized_contexts_anc_test_not_present])
print('Max idiom length: ', max_idiom_length_anc_test_not_present)
print('Max context length: ', max_context_length_anc_test_not_present)

padded_idioms_anc_test_not_present = pad_sentence(tokenized_idioms_anc_test_not_present, max_idiom_length_anc_test_not_present+5)
padded_contexts_anc_test_not_present = pad_sentence(tokenized_contexts_anc_test_not_present, max_context_length_anc_test_not_present+20)
print(len(padded_idioms_anc_test_not_present))
print(len(padded_contexts_anc_test_not_present))

masked_idioms_anc_test_not_present = create_att_masks(padded_idioms_anc_test_not_present)
masked_contexts_anc_test_not_present = create_att_masks(padded_contexts_anc_test_not_present)
print(len(masked_idioms_anc_test_not_present))
print(len(masked_contexts_anc_test_not_present))

embedded_idioms_anc_test_not_present = use_batches(padded_idioms_anc_test_not_present, masked_idioms_anc_test_not_present)
len(embedded_idioms_anc_test_not_present)

embedded_contexts_anc_test_not_present = use_batches(padded_contexts_anc_test_not_present, masked_contexts_anc_test_not_present)
len(embedded_contexts_anc_test_not_present)

Max idiom length:  11
Max context length:  192

Padding/truncating all sentences to 16 values...

Padding token: "[PAD]", ID: 0

Done.

Padding/truncating all sentences to 212 values...

Padding token: "[PAD]", ID: 0

Done.
163
163
163
163
0
NUM_OF_IDIOMS -= BATCH_SIZE -37
0
NUM_OF_IDIOMS -= BATCH_SIZE -37


163

In [ ]:
labels_anc_test_not_present = to_categorical(data_anc_test.Label)

### All

In [ ]:
tokenized_idioms_all_train_not_present = data_all_train['Idiom Inflected'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True, truncation=True, max_length=50)))
tokenized_contexts_all_train_not_present = data_all_train['Example'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True, truncation=True, max_length=500)))
max_idiom_length_all_train_not_present = max([len(sen) for sen in tokenized_idioms_all_train_not_present])
max_context_length_all_train_not_present = max([len(sen) for sen in tokenized_contexts_all_train_not_present])
print('Max idiom length: ', max_idiom_length_all_train_not_present)
print('Max context length: ', max_context_length_all_train_not_present)

padded_idioms_all_train_not_present = pad_sentence(tokenized_idioms_all_train_not_present, max_idiom_length_all_train_not_present+5)
padded_contexts_all_train_not_present = pad_sentence(tokenized_contexts_all_train_not_present, max_context_length_all_train_not_present+20)
print(len(padded_idioms_all_train_not_present))
print(len(padded_contexts_all_train_not_present))

masked_idioms_all_train_not_present = create_att_masks(padded_idioms_all_train_not_present)
masked_contexts_all_train_not_present = create_att_masks(padded_contexts_all_train_not_present)
print(len(masked_idioms_all_train_not_present))
print(len(masked_contexts_all_train_not_present))

embedded_idioms_all_train_not_present = use_batches(padded_idioms_all_train_not_present, masked_idioms_all_train_not_present)
len(embedded_idioms_all_train_not_present)

embedded_contexts_all_train_not_present = use_batches(padded_contexts_all_train_not_present, masked_contexts_all_train_not_present)
len(embedded_contexts_all_train_not_present)

Max idiom length:  11
Max context length:  234

Padding/truncating all sentences to 16 values...

Padding token: "[PAD]", ID: 0

Done.

Padding/truncating all sentences to 254 values...

Padding token: "[PAD]", ID: 0

Done.
1536
1536
1536
1536
0
NUM_OF_IDIOMS -= BATCH_SIZE 1336
200
NUM_OF_IDIOMS -= BATCH_SIZE 1136
400
NUM_OF_IDIOMS -= BATCH_SIZE 936
600
NUM_OF_IDIOMS -= BATCH_SIZE 736
800
NUM_OF_IDIOMS -= BATCH_SIZE 536
1000
NUM_OF_IDIOMS -= BATCH_SIZE 336
1200
NUM_OF_IDIOMS -= BATCH_SIZE 136
1400
NUM_OF_IDIOMS -= BATCH_SIZE -64
0
NUM_OF_IDIOMS -= BATCH_SIZE 1336
200
NUM_OF_IDIOMS -= BATCH_SIZE 1136
400
NUM_OF_IDIOMS -= BATCH_SIZE 936
600
NUM_OF_IDIOMS -= BATCH_SIZE 736
800
NUM_OF_IDIOMS -= BATCH_SIZE 536
1000
NUM_OF_IDIOMS -= BATCH_SIZE 336
1200
NUM_OF_IDIOMS -= BATCH_SIZE 136
1400
NUM_OF_IDIOMS -= BATCH_SIZE -64


1536

In [ ]:
labels_all_train_not_present = to_categorical(data_all_train.Label)

In [ ]:
tokenized_idioms_all_test_not_present = data_all_test['Idiom Inflected'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True, truncation=True, max_length=50)))
tokenized_contexts_all_test_not_present = data_all_test['Example'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True, truncation=True, max_length=500)))
max_idiom_length_all_test_not_present = max([len(sen) for sen in tokenized_idioms_all_test_not_present])
max_context_length_all_test_not_present = max([len(sen) for sen in tokenized_contexts_all_test_not_present])
print('Max idiom length: ', max_idiom_length_all_test_not_present)
print('Max context length: ', max_context_length_all_test_not_present)

padded_idioms_all_test_not_present = pad_sentence(tokenized_idioms_all_test_not_present, max_idiom_length_all_test_not_present+5)
padded_contexts_all_test_not_present = pad_sentence(tokenized_contexts_all_test_not_present, max_context_length_all_test_not_present+20)
print(len(padded_idioms_all_test_not_present))
print(len(padded_contexts_all_test_not_present))

masked_idioms_all_test_not_present = create_att_masks(padded_idioms_all_test_not_present)
masked_contexts_all_test_not_present = create_att_masks(padded_contexts_all_test_not_present)
print(len(masked_idioms_all_test_not_present))
print(len(masked_contexts_all_test_not_present))

embedded_idioms_all_test_not_present = use_batches(padded_idioms_all_test_not_present, masked_idioms_all_test_not_present)
len(embedded_idioms_all_test_not_present)

embedded_contexts_all_test_not_present = use_batches(padded_contexts_all_test_not_present, masked_contexts_all_test_not_present)
len(embedded_contexts_all_test_not_present)

Max idiom length:  11
Max context length:  192

Padding/truncating all sentences to 16 values...

Padding token: "[PAD]", ID: 0

Done.

Padding/truncating all sentences to 212 values...

Padding token: "[PAD]", ID: 0

Done.
349
349
349
349
0
NUM_OF_IDIOMS -= BATCH_SIZE 149
200
NUM_OF_IDIOMS -= BATCH_SIZE -51
0
NUM_OF_IDIOMS -= BATCH_SIZE 149
200
NUM_OF_IDIOMS -= BATCH_SIZE -51


349

In [ ]:
labels_all_test_not_present = to_categorical(data_all_test.Label)

## Train

### VNC

In [ ]:
model_vnc_not_present = Sequential()
model_vnc_not_present.add(Masking(mask_value=0., input_shape=(MAX_SEQUENCE_LEN,VECTOR_DIM)))
forward_layer = GRU(10, return_sequences=False, dropout=0.5)
backward_layer = GRU(10, return_sequences=False, dropout=0.5,
                    go_backwards=True)
model_vnc_not_present.add(Bidirectional(forward_layer, backward_layer=backward_layer,
                      input_shape=(MAX_SEQUENCE_LEN,VECTOR_DIM)))
model_vnc_not_present.add(Dense(NUM_CLASSES))
model_vnc_not_present.add(Activation('softmax'))

model_vnc_not_present.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
print('compiled model')

compiled model


In [ ]:
model_vnc_not_present.fit(np.asarray(embedded_contexts_vnc_train_not_present), 
              labels_vnc_train_not_present, batch_size=8, epochs=10)
print('fit model')

Epoch 1/10
112/112 [==============================] - 31s 217ms/step - loss: 0.6337 - accuracy: 0.6310
Epoch 2/10
112/112 [==============================] - 24s 218ms/step - loss: 0.4378 - accuracy: 0.8046
Epoch 3/10
112/112 [==============================] - 24s 217ms/step - loss: 0.4091 - accuracy: 0.8113
Epoch 4/10
112/112 [==============================] - 24s 218ms/step - loss: 0.3226 - accuracy: 0.8720
Epoch 5/10
112/112 [==============================] - 24s 217ms/step - loss: 0.3061 - accuracy: 0.8759
Epoch 6/10
112/112 [==============================] - 24s 216ms/step - loss: 0.2552 - accuracy: 0.8789
Epoch 7/10
112/112 [==============================] - 24s 215ms/step - loss: 0.2190 - accuracy: 0.9226
Epoch 8/10
112/112 [==============================] - 24s 216ms/step - loss: 0.1852 - accuracy: 0.9244
Epoch 9/10
112/112 [==============================] - 24s 216ms/step - loss: 0.1465 - accuracy: 0.9529
Epoch 10/10
112/112 [==============================] - 24s 217ms/step - l

In [ ]:
model_vnc_not_present.evaluate(np.asarray(embedded_contexts_vnc_test_not_present), labels_vnc_test_not_present)

6/6 [==============================] - 3s 72ms/step - loss: 0.1252 - accuracy: 0.9570


[0.12515968084335327, 0.9569892287254333]

In [ ]:
preds_vnc_not_present = model_vnc_not_present.predict(np.array(embedded_contexts_vnc_test_not_present))
f1_score(np.argmax(preds_vnc_not_present, axis=1), np.argmax(labels_vnc_test_not_present, axis=1))

0.943661971830986

### ANC

In [ ]:
model_anc_not_present = Sequential()
model_anc_not_present.add(Masking(mask_value=0., input_shape=(MAX_SEQUENCE_LEN,VECTOR_DIM)))
forward_layer = GRU(10, return_sequences=False, dropout=0.5)
backward_layer = GRU(10, return_sequences=False, dropout=0.5,
                    go_backwards=True)
model_anc_not_present.add(Bidirectional(forward_layer, backward_layer=backward_layer,
                      input_shape=(MAX_SEQUENCE_LEN,VECTOR_DIM)))
model_anc_not_present.add(Dense(NUM_CLASSES))
model_anc_not_present.add(Activation('softmax'))

model_anc_not_present.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
print('compiled model')

compiled model


In [ ]:
model_anc_not_present.fit(np.asarray(embedded_contexts_anc_train_not_present), 
              labels_anc_train_not_present, batch_size=8, epochs=10)
print('fit model')

Epoch 1/10
81/81 [==============================] - 23s 205ms/step - loss: 0.6829 - accuracy: 0.5610
Epoch 2/10
81/81 [==============================] - 17s 205ms/step - loss: 0.5093 - accuracy: 0.7728
Epoch 3/10
81/81 [==============================] - 17s 205ms/step - loss: 0.4509 - accuracy: 0.8079
Epoch 4/10
81/81 [==============================] - 17s 205ms/step - loss: 0.4196 - accuracy: 0.8005
Epoch 5/10
81/81 [==============================] - 17s 204ms/step - loss: 0.3624 - accuracy: 0.8470
Epoch 6/10
81/81 [==============================] - 17s 204ms/step - loss: 0.3158 - accuracy: 0.8774
Epoch 7/10
81/81 [==============================] - 17s 205ms/step - loss: 0.2712 - accuracy: 0.9077
Epoch 8/10
81/81 [==============================] - 17s 204ms/step - loss: 0.2395 - accuracy: 0.9062
Epoch 9/10
81/81 [==============================] - 17s 204ms/step - loss: 0.1902 - accuracy: 0.9419
Epoch 10/10
81/81 [==============================] - 17s 206ms/step - loss: 0.1378 - accura

In [ ]:
model_anc_not_present.evaluate(np.asarray(embedded_contexts_anc_test_not_present), labels_anc_test_not_present)

6/6 [==============================] - 3s 69ms/step - loss: 1.2682 - accuracy: 0.5644


[1.2682161331176758, 0.5644171833992004]

In [ ]:
preds_anc_not_present = model_anc_not_present.predict(np.array(embedded_contexts_anc_test_not_present))
f1_score(np.argmax(preds_anc_not_present, axis=1), np.argmax(labels_anc_test_not_present, axis=1))

0.6077348066298343

### All

In [ ]:
model_all_not_present = Sequential()
model_all_not_present.add(Masking(mask_value=0., input_shape=(MAX_SEQUENCE_LEN,VECTOR_DIM)))
forward_layer = GRU(10, return_sequences=False, dropout=0.5)
backward_layer = GRU(10, return_sequences=False, dropout=0.5,
                    go_backwards=True)
model_all_not_present.add(Bidirectional(forward_layer, backward_layer=backward_layer,
                      input_shape=(MAX_SEQUENCE_LEN,VECTOR_DIM)))
model_all_not_present.add(Dense(NUM_CLASSES))
model_all_not_present.add(Activation('softmax'))

model_all_not_present.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
print('compiled model')

compiled model


In [ ]:
model_all_not_present.fit(np.asarray(embedded_contexts_all_train_not_present), 
              labels_all_train_not_present, batch_size=8, epochs=10)
print('fit model')

Epoch 1/10
192/192 [==============================] - 33s 145ms/step - loss: 0.6494 - accuracy: 0.6351
Epoch 2/10
192/192 [==============================] - 28s 144ms/step - loss: 0.4975 - accuracy: 0.7652
Epoch 3/10
192/192 [==============================] - 28s 144ms/step - loss: 0.4495 - accuracy: 0.7829
Epoch 4/10
192/192 [==============================] - 27s 142ms/step - loss: 0.3941 - accuracy: 0.8253
Epoch 5/10
192/192 [==============================] - 27s 142ms/step - loss: 0.3771 - accuracy: 0.8389
Epoch 6/10
192/192 [==============================] - 28s 143ms/step - loss: 0.3301 - accuracy: 0.8648
Epoch 7/10
192/192 [==============================] - 28s 144ms/step - loss: 0.2957 - accuracy: 0.8809
Epoch 8/10
192/192 [==============================] - 28s 144ms/step - loss: 0.2910 - accuracy: 0.8754
Epoch 9/10
192/192 [==============================] - 27s 143ms/step - loss: 0.2493 - accuracy: 0.8928
Epoch 10/10
192/192 [==============================] - 28s 144ms/step - l

In [ ]:
model_all_not_present.evaluate(np.asarray(embedded_contexts_all_test_not_present), labels_all_test_not_present)

11/11 [==============================] - 3s 81ms/step - loss: 0.6098 - accuracy: 0.7765


[0.6098086833953857, 0.7765042781829834]

In [ ]:
preds_all_not_present = model_all_not_present.predict(np.array(embedded_contexts_all_test_not_present))
f1_score(np.argmax(preds_all_not_present, axis=1), np.argmax(labels_all_test_not_present, axis=1))

0.7845303867403315